In [20]:
import pandas as pd
import numpy as np

In [21]:
df=pd.read_csv("predictions.csv")
# df.drop(labels=["Unnamed: 0"], axis=1, inplace=True)
df.head()

,Unnamed: 0,prediction,source,stock_info,datetime
0,0,"Buy KEI Industries, target price Rs 5230",Motilal Oswal,"KEI Industries Ltd., incorporated in the year ...","Jul 31, 2024, 09:35 AM IST"
1,1,"Buy Hindustan Petroleum Corporation, target pr...",Motilal Oswal,"Hindustan Petroleum Corporation Ltd., incorpor...","Jul 31, 2024, 09:32 AM IST"
2,2,"Buy Tata Consumer Products, target price Rs 1380",Motilal Oswal,Tata Consumer Products Ltd. key Products/Reven...,"Jul 31, 2024, 09:28 AM IST"
3,3,"Buy Varun Beverages, target price Rs 1850",Motilal Oswal,"Varun Beverages Ltd., incorporated in the year...","Jul 31, 2024, 09:25 AM IST"
4,4,"Buy Indian Oil Corporation, target price Rs 215",Motilal Oswal,"Indian Oil Corporation Ltd., incorporated in t...","Jul 31, 2024, 09:22 AM IST"


In [22]:
d={'advice':[],'target':[],'stock':[]}
for ID,pred in df.iterrows():
    d["advice"].append(pred["prediction"].split()[0])
    d['target'].append(pred["prediction"].split()[-1])
    temp=pred["stock_info"].split(",")[0]
    d['stock'].append(temp.split(".")[0])

In [23]:
df["target"]=d["target"]
df["advice"]=d["advice"]
df["stock"]=d["stock"]

In [24]:
df["advice"].unique()

array(['Buy', 'Hold', 'Sell', 'Add', 'Reduce', 'Accumulate', 'Neutral'],
      dtype=object)

In [25]:
df["advice"].replace(to_replace=['Accumulate','Reduce','Add'],value=['buy','sell','buy'],inplace=True)

C:\Users\gupta\AppData\Local\Temp\ipykernel_25544\287399997.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["advice"].replace(to_replace=['Accumulate','Reduce','Add'],value=['buy','sell','buy'],inplace=True)


In [26]:
df.head()

,Unnamed: 0,prediction,source,stock_info,datetime,target,advice,stock
0,0,"Buy KEI Industries, target price Rs 5230",Motilal Oswal,"KEI Industries Ltd., incorporated in the year ...","Jul 31, 2024, 09:35 AM IST",5230,Buy,KEI Industries Ltd
1,1,"Buy Hindustan Petroleum Corporation, target pr...",Motilal Oswal,"Hindustan Petroleum Corporation Ltd., incorpor...","Jul 31, 2024, 09:32 AM IST",460,Buy,Hindustan Petroleum Corporation Ltd
2,2,"Buy Tata Consumer Products, target price Rs 1380",Motilal Oswal,Tata Consumer Products Ltd. key Products/Reven...,"Jul 31, 2024, 09:28 AM IST",1380,Buy,Tata Consumer Products Ltd
3,3,"Buy Varun Beverages, target price Rs 1850",Motilal Oswal,"Varun Beverages Ltd., incorporated in the year...","Jul 31, 2024, 09:25 AM IST",1850,Buy,Varun Beverages Ltd
4,4,"Buy Indian Oil Corporation, target price Rs 215",Motilal Oswal,"Indian Oil Corporation Ltd., incorporated in t...","Jul 31, 2024, 09:22 AM IST",215,Buy,Indian Oil Corporation Ltd


In [27]:
df["stock"].value_counts()

stock
Hindustan Petroleum Corporation Ltd    3
Varun Beverages Ltd                    3
Avenue Supermarts Ltd                  3
SBI Life Insurance Company Ltd         3
ICICI Bank Ltd                         3
                                      ..
Natco Pharma Ltd                       1
Syrma SGS Technology Ltd               1
Kaynes Technology India Ltd            1
Bharat Electronics Ltd                 1
ITC Ltd                                1
Name: count, Length: 182, dtype: int64

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


def get_stock_code(stock_name):

    path="https://finance.yahoo.com/quote/0A7M.L/"

    chrome_options = Options()
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--headless')

    driver=webdriver.Chrome(options=chrome_options)

    driver.get(path)

    time.sleep(3)
    search = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ybar-sbq"))
        )
    search.send_keys(stock_name)

        # Click on the search button
    submit = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "ybar-search"))
        )
    submit.click()
    time.sleep(2)
    try:
        stock_code = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME,"yf-3a2v0c"))
            ).text
    except Exception:
        return False

    return stock_code

codes={}
for stock_name in df["stock"].unique():
    if (df["stock"] == stock_name).sum() > 0:
        stock_code = get_stock_code(stock_name=stock_name)
        if stock_code:
            stock_code=stock_code.split(" (")[1].split(")")[0]
            print(stock_code)
            codes[stock_name]=stock_code

In [ ]:
codes

{}

In [ ]:
filtered_df=df[(df["stock"].isin(list(codes.keys())))]# & (df["datetime"]<"Jul 28, 2024, 09:39 AM IST")]

filtered_df.to_csv("refined_predictions.csv",index=False)

### Fetching Real World Data from Yahoo Finance through API

In [ ]:
import yfinance as yahooFinance
import pandas as pd
import os

if "stock_dataset" not in os.listdir():
    os.makedirs("./stock_dataset")

for stock in codes:
    data=yahooFinance.Ticker(codes[stock])
    df=pd.DataFrame(data.history(period="max"))
    df.index=df.index.date
    df.to_csv(f"stock_dataset/{stock}.csv")

In [ ]:
# import yfinance as yahooFinance
 
# Tata_Motors = yahooFinance.Ticker("TATAMOTORS.NS")
# Ashok_Leyland = yahooFinance.Ticker("ASHOKLEY.NS")
# Mahindra = yahooFinance.Ticker("M&M.NS")
# Reliance = yahooFinance.Ticker("RELIANCE.NS")
# Maruti = yahooFinance.Ticker("MARUTI.NS")
# hdfc = yahooFinance.Ticker("HDFCBANK.NS")
# icici = yahooFinance.Ticker("ICICIBANK.NS")
# axis = yahooFinance.Ticker("AXISBANK.NS")
# sbi = yahooFinance.Ticker("SBIN.NS")
# hidalco = yahooFinance.Ticker("HINDALCO.BO")

# # Let us  get historical stock prices for Facebook 
# # covering the past few years.
# # max->maximum number of daily prices available 
# # for Facebook.
# # Valid options are 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 
# # 5y, 10y and ytd.

# import pandas as pd

# df=pd.DataFrame(Maruti.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Maruti Suzuki India Ltd.csv")
# df=pd.DataFrame(sbi.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/State Bank of India.csv")
# df=pd.DataFrame(axis.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Axis Bank Ltd.csv")
# df=pd.DataFrame(icici.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/ICICI Bank Ltd.csv")
# df=pd.DataFrame(Reliance.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Reliance Industries Ltd.csv")
# df=pd.DataFrame(Mahindra.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Mahindra & Mahindra Ltd.csv")
# df=pd.DataFrame(Ashok_Leyland.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Ashok Leyland Ltd.csv")
# df=pd.DataFrame(Tata_Motors.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Tata Motors Ltd.csv")
# df=pd.DataFrame(hdfc.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/HDFC Bank Ltd.csv")
# df=pd.DataFrame(hidalco.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Hindalco Industries Ltd.csv")
# df=pd.DataFrame(hidalco.history(period="3y"))
# df.index=df.index.date
# df.to_csv("stock_dataset/Hindalco Industries Ltd.csv")


### Convert Stock Data To JSON format

In [ ]:
# import json
# import os
# import pandas as pd

# files=os.listdir("stock_dataset")
# # print(files)

# data=[]
# counter=1

# for stock in files:
#     d={"data":[]}
#     df=pd.read_csv(f"stock_dataset/{stock}")
#     name=stock.split(".")[0]
#     d["name"]=name
#     d["id"]=counter
#     for ID,VALUE in df.iterrows():
#         d['data'].append({"date":VALUE["Unnamed: 0"],"value":VALUE.Close})
#     counter+=1
#     data.append(d)
        
# stocks_data=json.dumps(data,indent=2)

# with open("Market_Stocks_Data", "w") as json_file:
#     json_file.write(stocks_data)

# print(f"JSON data has been saved to Market_Stocks_Data")

### Calculating Credibility Scores

In [ ]:
import pandas as pd

preds=pd.read_csv("refined_predictions.csv")
# preds.head()

preds.source.value_counts()

Series([], Name: count, dtype: int64)

In [ ]:
def CRED_SCORE(preds2):    

    from datetime import date, timedelta
    from sklearn.metrics import mean_absolute_percentage_error

    PREDICTIONS=[]
    ACTUAL=[]
    predictions=[]
    for ID, DATA in preds2.iterrows():

        if (date.today() - DATA['datetime']) > timedelta(days=30):
            name=DATA.source

            df = pd.read_csv(f"stock_dataset/{DATA['stock']}.csv")
            df.index = pd.to_datetime(df["Unnamed: 0"])
            df.index=df.index.date

            start_date = DATA['datetime']
            end_date = start_date + timedelta(days=7)

            df = df[(df.index >= start_date) & (df.index <= end_date)]

            if DATA['advice'] == "Buy":
                change = float(max(df['Close'])-df[df.index==DATA.datetime]['Close'])
                PREDICTIONS.append(change)
                ACTUAL.append(DATA.target)
            else:
                change = float(-min(df['Close'])+df[df.index==DATA.datetime]['Close'])
                PREDICTIONS.append(change)
                ACTUAL.append(DATA.target)

        if  (date.today() - DATA['datetime']) < timedelta(days=14):
            d2={ "stockInfo": DATA.stock_info,
            "date": DATA.datetime.strftime('%Y-%m-%d'),
            "targetValue": DATA.target,
            "advice": DATA.advice}
            predictions.append(d2)
            
    if (not ACTUAL) or (not PREDICTIONS):
        return {"score":"Not Available","predictions":predictions}
    credibility=abs(mean_absolute_percentage_error(ACTUAL,PREDICTIONS))*10

    return {"score":credibility.round(2),"predictions":predictions}

In [ ]:
import json
from langchain_ollama.llms import OllamaLLM

counter=0
json_file=[]

model = OllamaLLM(model="llama3") 

for person in preds["source"].unique():
    
    d={}

    preds2=preds[preds["source"]==person]
    preds2.datetime=pd.to_datetime(preds2.datetime)
    preds2.datetime=preds2.datetime.dt.date
    
    d["id"]= counter
    d["name"]= preds2.iloc[0].source
    d["description"]=model.invoke(f"give a brief discription of {preds2.iloc[0].source} in one paragraph")

    d2=CRED_SCORE(preds2)
    d["credibilityScore"]=d2["score"]
    d["predictions"]=d2["predictions"]

    json_file.append(d)

    counter+=1

# print(json_file)

json_file=json.dumps(json_file,indent=2)

with open("Market_Predictions", "w") as js_file:
    js_file.write(json_file)

print(f"JSON data has been saved to Market_Predictions")

JSON data has been saved to Market_Predictions
